In [ ]:
### Use this dataset to construct heat budget climatologies – we currently cannot provide the code to make the heat budget, as we await Ryan Holmes' submission

In [1]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import cftime
from tqdm import tqdm
import cmocean as cm
import sys, os
from datetime import datetime

from dask.distributed import Client

In [2]:
from dask.distributed import Client

client = Client(threads_per_worker=1)
client

/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38577 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38577/status,
Dashboard: /proxy/38577/status,Workers: 21
Total threads: 21,Total memory: 765.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45283,Workers: 21
Dashboard: /proxy/38577/status,Total threads: 21
Started: Just now,Total memory: 765.00 GiB
Comm: tcp://127.0.0.1:43271,Total threads: 1
Dashboard: /proxy/46503/status,Memory: 36.43 GiB
Nanny: tcp://127.0.0.1:36983,


## Mixed layer heat budget output load in

In [3]:
import xarray as xr

file_list = [
    f'/g/data/e14/ts7341/Figure_Code_For_Manuscript/Science/JRA_SST_Processed/Heat_Budget_Data/full_mixed_layer_heat_budget_output{idx}.nc'
    for idx in range(329, 364)  
]

ds = xr.open_mfdataset(file_list, combine='by_coords')
ds = ds.sel(yt_ocean = slice(-78,-70)).sel(xt_ocean=slice(-210,-120))

In [4]:
data_sample = ds

In [5]:
data_sample['time'] = data_sample['time'] - np.timedelta64(12, 'h') - np.timedelta64(1, 'D')

In [6]:
print(data_sample)

<xarray.Dataset> Size: 56GB
Dimensions:                 (xt_ocean: 360, yt_ocean: 75, time: 12748)
Coordinates:
  * xt_ocean                (xt_ocean) float64 3kB -209.9 -209.6 ... -120.1
  * yt_ocean                (yt_ocean) float64 600B -77.91 -77.8 ... -70.09
  * time                    (time) datetime64[ns] 102kB 1980-12-31T12:00:00 ....
Data variables: (12/21)
    fixedh_tendency         (time, yt_ocean, xt_ocean) float64 3GB dask.array<chunksize=(364, 75, 360), meta=np.ndarray>
    temp_adv                (time, yt_ocean, xt_ocean) float64 3GB dask.array<chunksize=(364, 75, 360), meta=np.ndarray>
    submeso                 (time, yt_ocean, xt_ocean) float64 3GB dask.array<chunksize=(364, 75, 360), meta=np.ndarray>
    neut_diffusion          (time, yt_ocean, xt_ocean) float64 3GB dask.array<chunksize=(364, 75, 360), meta=np.ndarray>
    neutral_gm              (time, yt_ocean, xt_ocean) float64 3GB dask.array<chunksize=(364, 75, 360), meta=np.ndarray>
    temp_vdiffuse_k33     

In [7]:
## Have to perform this step for the data to work below

def _to_da(obj, varname=None):
    """
    Return a DataArray from either a DataArray or Dataset.
    If varname is None and obj is a Dataset, pick the first data_var that has 'time' in dims.
    """
    if isinstance(obj, xr.DataArray):
        return obj
    if isinstance(obj, xr.Dataset):
        if varname is not None:
            return obj[varname]
        # pick first var with 'time' in dims
        for v in obj.data_vars:
            if 'time' in obj[v].dims:
                return obj[v]
        # fallback: first variable
        return next(iter(obj.data_vars.values()))
    raise TypeError("Expected xarray DataArray or Dataset")


In [8]:
def smoothedClima_mhw(obj, varname=None, smoothPercentile=True, smoothPercentileWidth=31):
    """
    Climatology (DOY) smoothed with 31-day running mean.
    Accepts DataArray or Dataset. If Dataset, pass varname or the first time-varying var is used.
    Returns a DataArray (same spatial dims, 'dayofyear' dim).
    """
    da = _to_da(obj, varname)
    if "time" not in da.dims:
        raise ValueError("No 'time' dimension found.")

    clim = da.groupby("time.dayofyear").mean()

    # If user requests smoothing:
    if smoothPercentile:
        # For optimized concatenation, create doy vector with just the right amount of time steps
        year = 1996 # Dummy year. The reference year needs to be a leap year
        # Create a pandas datetime of 366 dayof year
        date_doy = pd.to_datetime(clim.dayofyear - 1, unit='D', origin=f'{year}-01-01', errors='coerce')
        # Concat the last 31 days, the whole year and the first 31 days (for datetime vector and for data)
        time_concat = [*(date_doy[-smoothPercentileWidth:]- pd.DateOffset(years=1)), 
                       *date_doy, 
                       *(date_doy[:smoothPercentileWidth] + pd.DateOffset(years=1))]
        stacked = xr.concat([clim.isel(dayofyear=slice(-smoothPercentileWidth,None)), 
                             clim, 
                             clim.isel(dayofyear=slice(None,smoothPercentileWidth))], dim='dayofyear') \
                    .assign_coords({'time':('dayofyear',time_concat)}) \
                    .swap_dims({'dayofyear':'time'})

        # PERF: ensure stacked core dim is single-chunk for rolling
        if hasattr(stacked.data, "chunks"):
            stacked = stacked.chunk({"time": -1})
        # The actual smoothing    
        smoothed = stacked.rolling(time=smoothPercentileWidth, min_periods=1, center=True).mean()
        # Now extract the middle year using this little groupby trick. Another way would be to isel(doy=slice())
        mid_sel = smoothed.groupby('time.year')[1996].swap_dims({'time':'dayofyear'}).drop_vars('time')
        # Make sure the names are right
        out = mid_sel.rename({'time':'dayofyear'}) if 'time' in mid_sel.dims else mid_sel
    else: # If no smoothing
        out = clim
    return out.assign_coords(dayofyear=clim.dayofyear.data)


In [9]:
## Baseline of 1982 until 2012 selected
clim_time = slice('1982-01-01','2011-12-31')
clim_sample = data_sample.sel(time = clim_time)

clim_sample

<xarray.Dataset> Size: 48GB
Dimensions:                 (xt_ocean: 360, yt_ocean: 75, time: 10927)
Coordinates:
  * xt_ocean                (xt_ocean) float64 3kB -209.9 -209.6 ... -120.1
  * yt_ocean                (yt_ocean) float64 600B -77.91 -77.8 ... -70.09
  * time                    (time) datetime64[ns] 87kB 1982-01-01T12:00:00 .....
Data variables: (12/21)
    fixedh_tendency         (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    temp_adv                (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    submeso                 (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    neut_diffusion          (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    neutral_gm              (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    temp_vdiffuse_k33       (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    ...                      ...
    sens_heat               (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    evap_heat               (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    sw_heat_in_mld          (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    residual                (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    mlt_tendency            (time, yt_ocean, xt_ocean) float32 1GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    entrainment             (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
Attributes:
    Project:   Antarctic Seasonal Ice Zone MHWs 1982-2019 (plus extension to ...
    Author:    Tom Schmaltz - Hons Student 2025
    Data set:  ACCESS-OM2-025 IAF simulation forced by JRA55 (cycle6)
    Created:   2025-10-20
    Mail:      t.schmaltz@student.unsw.edu.au

In [10]:
print(clim_sample)

<xarray.Dataset> Size: 48GB
Dimensions:                 (xt_ocean: 360, yt_ocean: 75, time: 10927)
Coordinates:
  * xt_ocean                (xt_ocean) float64 3kB -209.9 -209.6 ... -120.1
  * yt_ocean                (yt_ocean) float64 600B -77.91 -77.8 ... -70.09
  * time                    (time) datetime64[ns] 87kB 1982-01-01T12:00:00 .....
Data variables: (12/21)
    fixedh_tendency         (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    temp_adv                (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    submeso                 (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    neut_diffusion          (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    neutral_gm              (time, yt_ocean, xt_ocean) float64 2GB dask.array<chunksize=(363, 75, 360), meta=np.ndarray>
    temp_vdiffuse_k33     

In [1]:
import xarray as xr
import os

## Loop through each variable

output_file = "/scratch/e14/ts7341/Climatology_RS.nc"

all_vars = list(clim_sample.data_vars)

# Loop variables
for i, var in enumerate(all_vars):
    if var in existing_vars:
        print(f"Skipping already processed variable: {var}")
        continue


    try:
        var_data = clim_sample[[var]].load()

        # Smooth
        smoothed_var = smoothedClima_mhw(var_data)

        arr = smoothed_var.compute()

        ds_temp = arr.to_dataset(name=var)

        # Compress
        encoding = {var: {'zlib': True, 'complevel': 4}}

        # Write out
        ds_temp.to_netcdf(
            output_file,
            mode='w' if (i == 0 and len(existing_vars) == 0) else 'a',
            engine='h5netcdf',
            encoding=encoding
        )

        del arr, ds_temp, smoothed_var, var_data


SyntaxError: incomplete input (3995241662.py, line 36)

In [ ]:
## Ensure that it works
ds = xr.open_dataset('/g/data/e14/ts7341/Figure_Code_For_Manuscript/Science/Heat_Budget_Data/Climatology.nc')
ds